In [ ]:
from cahos import Scale, VoiceLeading, get_scales
import polars as pl

In [ ]:
dis_subseq = [
    [1, 2], [2, 1], [3, 1],
    # [3, 4], [4, 3],
    [6, 1], [1, 6], [5, 1], [1, 5],
    [1, 1], [2, 2],  # [3, 3], [4, 4],
    [5, 5, 5], [6, 6, 6], [7, 7, 7],
]

# 80 is the maximum span we can get with the Cahos ensemble
# Will drop bass by an octave, hence `-12`
base_chords = [Scale(s) for s in get_scales(
    allowed_intervals=[1, 2, 3, 4, 5, 6, 7],
    disallowed_subsequences=dis_subseq,
    disallowed_beginnings=[],
    max_span=80-12
)]

print(f"number of scales satisfying given constraints: {len(base_chords)}")

In [ ]:
df_base_chords = pl.DataFrame([{
    "deltas": s.deltas,
    "span": s.span,
    "sequence_entropy": s.sequence_entropy,
    "n_intervals": s.n_intervals
} for s in base_chords])

print(df_base_chords)

In [ ]:
df_selection = df_base_chords.filter(
    pl.col("n_intervals") == 3
).sort(["sequence_entropy", "span"], descending=True)
print(df_selection)

In [ ]:
n_shared_notes = 5

b1 = 40
b2 = 42
voice_leading_opportunities = []
for c1 in df_selection.rows():
    for c2 in df_selection.rows():
        vl = VoiceLeading(Scale(c1[0]), b1, Scale(c2[0]), b2)
        if max(vl.real_a + vl.real_b) > 108 or min(vl.real_a + vl.real_b) < 28 or vl.motion_balance() < 3/4 or vl.n_swaps() > 0:
            continue
        if vl.n_common_notes() == n_shared_notes and vl.max_step_size() < 3 and vl.n_pseudo_changes() == 0:
            voice_leading_opportunities.append(vl)

In [ ]:
df_vl = pl.DataFrame([{
    "left": vl.real_a_names,
    "right": vl.real_b_names
} for vl in voice_leading_opportunities])
print(df_vl)